In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
df_item[df_item['type']=='SingleQuestion']['variable_name'].value_counts()

In [ ]:
df_item['type'].value_counts()

In [ ]:
  
       'f__time_changed', 'f__answer_changed', 
       'f__answer_removed' ]

In [ ]:
pivot_table, index_col = self.get_clean_pivot_table('f__answer_selected',remove_low_freq_col=True)

In [ ]:
pivot_table

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns[0:14]:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    fig = px.box(data, x='responsible', y=col)
    fig.show()

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    pivot_table[col+'answer_set_anomaly'] = False
    lower_outlier, upper_outlier = get_outlier_iqr(data, col)
    pivot_table.loc[(~pd.isnull(pivot_table[col])) &((lower_outlier)|(upper_outlier)), col+'answer_set_anomaly'] = True

In [ ]:
selected_columns = pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns
selected_columns = [col for col in selected_columns if 'answer_set_anomaly' in col]
pivot_table['total_selected_columns'] = pivot_table[selected_columns].sum(1)

In [ ]:
pivot_table['total_selected_columns'].value_counts()

In [ ]:
pivot_table.loc[pivot_table['total_selected_columns']>4,'total_selected_columns'] =4
pivot_table['total_selected_columns'].hist()